In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(r"db\market_place.db")

cursor = conn.cursor()

In [2]:
aisles = pd.read_csv(r"data\aisles.csv", sep=",")
departments = pd.read_csv(r"data\departments.csv", sep=",")
products = pd.read_csv(r"data\products.csv", sep=",")
orders = pd.read_csv(r"data\orders_with_nulls.csv", sep=",")

print(aisles.isna().sum(), "\n")
print(departments.isna().sum(), "\n")
print(products.isna().sum(), "\n")
print(orders.isna().sum(), "\n")

aisle_id    0
aisle       0
dtype: int64 

department_id    0
department       0
dtype: int64 

product_id       0
product_name     0
aisle_id         0
department_id    0
dtype: int64 

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64 



In [3]:
# Calcular la media de 'days_since_prior_order' por usuario
mean_days_per_user = orders.groupby('user_id')['days_since_prior_order'].transform('mean')

# Reemplazar los valores NaN con la media de cada usuario
orders['days_since_prior_order'] = orders['days_since_prior_order'].fillna(mean_days_per_user).astype(int)

orders.to_csv("data\orders_not_nulls.csv", index=False)

<>:7: SyntaxWarning: invalid escape sequence '\o'
<>:7: SyntaxWarning: invalid escape sequence '\o'
C:\Users\cesar\AppData\Local\Temp\ipykernel_14692\686205838.py:7: SyntaxWarning: invalid escape sequence '\o'
  orders.to_csv("data\orders_not_nulls.csv", index=False)


In [5]:
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS aisles (
        aisle_id INT PRIMARY KEY,
        aisle_name TEXT NOT NULL
    )
    """
)

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS departments (
        department_id INT PRIMARY KEY,
        department_name TEXT NOT NULL
    )
    """
)

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS products (
        product_id INT PRIMARY KEY,
        product_name TEXT NOT NULL,
        aisle_id INT NOT NULL,
        department_id INT NOT NULL,
        FOREIGN KEY (aisle_id) REFERENCES aisles(aisle_id),
        FOREIGN KEY (department_id) REFERENCES departments(department_id)
    )
    """
)

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS users (
        user_id INT PRIMARY KEY,
        n_compras INT NOT NULL,
        last_order_date INT NOT NULL,
        mean_time_bt_orders INT NOT NULL,
        segment TEXT NOT NULL
    )
    """
)

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS orders (
        order_id INT NOT NULL,
        user_id INT NOT NULL,
        product_id INT NOT NULL,
        add_to_cart_order INT NOT NULL,
        reordered INT NOT NULL,
        order_number INT NOT NULL,
        order_dow INT NOT NULL,
        order_hour_of_day INT NOT NULL,
        days_since_prio_order INT NOT NULL,
        PRIMARY KEY (order_id,user_id,product_id),
        FOREIGN KEY (product_id) REFERENCES products(product_id)
    )
    """
)

conn.commit()

In [ ]:
# Inserta los datos en la base de datos
users = pd.read_csv(r"data\segmented_users.csv", sep=",")

# PASAR A MINUSCULAS TODOS LOS NOMBRES DE PRODUCTOS
products['product_name'] = products['product_name'].str.lower()
products['product_name'] = products['product_name'].str.replace('"', '', regex=False)
products['product_name'] = products['product_name'].str.replace(r"[^a-z0-9' ]", "", regex=True)
products = products.drop_duplicates(subset=['product_name', 'aisle_id', 'department_id'], keep='first')


aisles.to_sql("aisles", conn, if_exists="append", index=False)
departments.to_sql("departments", conn, if_exists="append", index=False)
products.to_sql("products", conn, if_exists="append", index=False)
users.to_sql("users", conn, if_exists="append", index=False)

chunk_size = 10000
total = len(orders)
for i in range(0, total, chunk_size):
    chunk = orders.iloc[i:i+chunk_size]
    chunk.to_sql("orders", conn, if_exists="append", index=False)
    print(f"Inserted rows {i} to {i + len(chunk) - 1}")




Inserted rows 0 to 9999
Inserted rows 10000 to 19999
Inserted rows 20000 to 29999
Inserted rows 30000 to 39999
Inserted rows 40000 to 49999
Inserted rows 50000 to 59999
Inserted rows 60000 to 69999
Inserted rows 70000 to 79999
Inserted rows 80000 to 89999
Inserted rows 90000 to 99999
Inserted rows 100000 to 109999
Inserted rows 110000 to 119999
Inserted rows 120000 to 129999
Inserted rows 130000 to 139999
Inserted rows 140000 to 149999
Inserted rows 150000 to 159999
Inserted rows 160000 to 169999
Inserted rows 170000 to 179999
Inserted rows 180000 to 189999
Inserted rows 190000 to 199999
Inserted rows 200000 to 209999
Inserted rows 210000 to 219999
Inserted rows 220000 to 229999
Inserted rows 230000 to 239999
Inserted rows 240000 to 249999
Inserted rows 250000 to 259999
Inserted rows 260000 to 269999
Inserted rows 270000 to 279999
Inserted rows 280000 to 289999
Inserted rows 290000 to 299999
Inserted rows 300000 to 309999
Inserted rows 310000 to 319999
Inserted rows 320000 to 329999
In

: 